In [0]:
# Alteração
from pyspark.sql.functions import current_timestamp, lit

# 1. Definir o caminho base para a Landing Zone e o novo Schema
CATALOG_NAME = "workspace" # Seu catálogo principal
SCHEMA_NAME = "projeto_medalhao_ecommerce" # O novo schema que você criou
LANDING_PATH = f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/landing_zone/"

# Lista dos 5 arquivos para processar
files_to_process = ["customers.csv", "order_items.csv", "orders.csv", "product_reviews.csv", "products.csv"]

for file_name in files_to_process:
    table_name = file_name.replace(".csv", "") # Ex: 'customers'
    
    # 2. Leitura do CSV (Landing Zone)
    df_raw = (spark.read.format("csv")
                .option("header", "true") 
                .option("inferSchema", "true") # Inferir os tipos de dados
                .load(f"{LANDING_PATH}{file_name}"))

    # 3. Enriquecimento: Adicionar metadados e marcar como 'Bronze'
    df_bronze = (df_raw
                 .withColumn("source_file", lit(file_name))
                 .withColumn("ingestion_timestamp", current_timestamp()))
    
    # 4. Escrita na Camada Bronze como Tabela Delta
    # A tabela será salva como 'projeto_medalhao_ecommerce.bronze_customers'
    (df_bronze.write.format("delta")
              .mode("overwrite") # Use 'overwrite' para o primeiro teste ou 'append' se for ingestão contínua
              .saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.bronze_{table_name}"))
              
    print(f"Tabela {SCHEMA_NAME}.bronze_{table_name} criada com sucesso.")